# Final Evaluation 

In this notebook the final resut is going to be evaluated. The data is first loaded into the Notebokk and then loaded into the current database.
After that the pipeline in the scope of this thesis is applied and the results collected for final evaluation

#### Loading the data

In [1]:
import pandas
import numpy
import re
path="Data/itunes_amazon_raw_data/labeled_data.csv"

#Read the file
data=pandas.read_csv(path, skiprows=5, encoding='unicode_escape')
data


,Unnamed: 0,_id,ltable.Sno,rtable.Sno,ltable.Album_Name,ltable.Artist_Name,ltable.CopyRight,ltable.Released,ltable.Song_Name,ltable.Time,rtable.Album_Name,rtable.Artist_Name,rtable.CopyRight,rtable.Released,rtable.Song_Name,rtable.Time,label
0,916,916,111,53124,vhs,x ambassadors,2015 kidinakorner/interscope records,30-Jun-15,vhs outro (interlude),1:25,vhs [explicit],x ambassadors,(c) 2015 kidinakorner/interscope records,"June 30, 2015",vhs outro (interlude) [explicit],1:25,1
1,1053,1053,148,50767,title (deluxe),meghan trainor,"2014, 2015 epic records, a division of sony m...",9-Jan-15,credit,2:51,title (deluxe),meghan trainor,"2011 what a music ltd, licence exclusive parl...","January 9, 2015",credit,2:51,1
2,1290,1290,206,41214,slow down (remixes),selena gomez,"2013 hollywood records, inc.",20-Aug-13,slow down (smash mode remix),5:21,slow down remixes,selena gomez,"(c) 2013 hollywood records, inc.","August 20, 2013",slow down (smash mode remix),5:21,1
3,1424,1424,211,19812,slow down (reggae remixes) - single,selena gomez,"2013 hollywood records, inc.",20-Aug-13,slow down (sure shot rockers reggae dub remix),3:15,good for you (remixes),selena gomez,(c) 2015 interscope records,"September 4, 2015",good for you (yellow claw & cesqeaux remix) [f...,3:01,0
4,1706,1706,250,53111,vhs,x ambassadors,2015 kidinakorner/interscope records,30-Jun-15,vhs outro (interlude),1:25,vhs [explicit],x ambassadors,(c) 2015 kidinakorner/interscope records,"June 30, 2015",first show (interlude),0:11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,57960,57960,6867,13598,doggumentary,snoop dogg,"__æ 2011 capitol records, llc. all rights re...",29-Mar-11,peer pressure (feat. traci nelson),4:07,doggumentary [edited],snoop dogg,"(c) 2011 capitol records, llc","March 29, 2011",boom ((feat. t-pain) [edited]),3:50,0
535,57993,57993,6868,13597,doggumentary,snoop dogg,"__æ 2011 capitol records, llc. all rights re...",29-Mar-11,i don't need no b**** (feat. devin the dude & ...,3:59,doggumentary [edited],snoop dogg,"(c) 2011 capitol records, llc","March 29, 2011",platinum ((feat. r. kelly) [edited]),4:29,0
536,58289,58289,6877,13623,doggumentary,snoop dogg & gorillaz,"__æ 2011 capitol records, llc. all rights re...",29-Mar-11,sumthin like this night (feat. gorillaz),3:37,doggumentary [explicit],snoop dogg,"(c) 2011 capitol records, llc","March 29, 2011",this weed iz mine (feat. wiz khalifa) [explicit],3:43,0
537,58403,58403,6882,13611,doggumentary,snoop dogg & latoiya williams,"_Ê 2011 capitol records, llc. all rights re...",29-Mar-11,cold game (feat. latoiya williams),3:49,doggumentary [explicit],snoop dogg,"(c) 2011 capitol records, llc","March 29, 2011",toyz n da hood (feat. bootsy collins) [explicit],2:40,0


In [188]:
class_vector=data['label']
left_table=data.iloc[:,4:10]
right_table=data.iloc[:,10:16]

In [189]:
#Import the necessary parts from the pipeline
from Evaluation.evaluation import evaluate_results
from Main.combined_pipeline import combined_pipeline
from Evaluation.test_evaluation import evaluate_results
from Utilities.database import query_database

In [190]:
#Function for populating the database
def create_and_populate_table(dataframe, table_name):
    try:
        # Clean table name (optional)
        table_name = table_name.replace(" ", "_")

        # Drop table if it exists
        delete_table_query = f"DROP TABLE IF EXISTS {table_name} CASCADE;"
        query_database(delete_table_query, printing=False)

        # Convert all data to string format
        dataframe = dataframe.astype(str)

        # Create table with a single column
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (Aggregate TEXT);"
        query_database(create_table_query, printing=False)
        print(f"Table '{table_name}' created successfully.")

        # Insert data into table
        values = []
        for _, row in dataframe.iterrows():
            row_values = "\n".join(row.astype(str))  # Concatenate row values with newline separator
            values.append(f"('{row_values.replace("'", "''")}')")  # Escape single quotes for SQL

        if values:
            insert_query = f"INSERT INTO {table_name} VALUES {', '.join(values)};"
            query_database(insert_query, printing=False)
            print(f"Table '{table_name}' populated successfully.")
        else:
            print("No data to insert.")

    except Exception as e:
        print(f"An error occurred: {e}")

In [191]:
#Define the index range
index_start = 0
index_end = 30

execution_mode = "local"
#execution_mode = "global"
#Running locally
if execution_mode == "local":
    create_and_populate_table(left_table.iloc[index_start:index_end, :], "left_table")
    create_and_populate_table(right_table.iloc[index_start:index_end, :], "right_table")
    ground_truth_left = data.iloc[index_start:index_end, ]
    ground_truth_left = ground_truth_left[ground_truth_left["label"] == 1]
    ground_truth_left = ground_truth_left.iloc[:, 4:10]
    

#4-10
else: 
    create_and_populate_table(left_table, "left_table")
    create_and_populate_table(right_table, "right_table")
    ground_truth_left = data
    ground_truth_left = ground_truth_left[ground_truth_left["label"] == 1]
    ground_truth_left = ground_truth_left.iloc[:, 4:10]



create_and_populate_table(ground_truth_left, "ground_truth_left")
ground_truth_answer=query_database("SELECT * FROM ground_truth_left;", printing=False)
ground_truth_answer

Table 'left_table' created successfully.
Table 'left_table' populated successfully.
Table 'right_table' created successfully.
Table 'right_table' populated successfully.
Table 'ground_truth_left' created successfully.
Table 'ground_truth_left' populated successfully.


[('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',),
 ('title (deluxe)\nmeghan trainor\n 2014, 2015 epic records, a division of sony music entertainment\n9-Jan-15\ncredit\n2:51',),
 ('slow down (remixes)\nselena gomez\n 2013 hollywood records, inc.\n20-Aug-13\nslow down (smash mode remix)\n5:21',),
 ('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',),
 ('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',),
 ('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',),
 ('the reason - ep\nx ambassadors\n 2015 kidinakorner/interscope records\n10-Aug-15\nshining\n3:40',)]

In [192]:
from Main.combined_pipeline import combined_pipeline
answer, metadata= combined_pipeline(query=None,initial_sql_query="SELECT * FROM right_table JOIN left_table ON left_table.aggregate = right_table.aggregate;",aux=True)

The final answer to the query is []
--------------------------------------
The 
SELECT * FROM right_table JOIN left_table ON left_table.aggregate = right_table.aggregate;
 has a JOIN clause.
[['SELECT aggregate FROM left_table;', "(2, <Comparison '=' at 0x7443503C1D20>)", 'left_table.aggregate = right_table.aggregate', 'SELECT aggregate FROM right_table;']]
The final answer to the query is [('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',), ('title (deluxe)\nmeghan trainor\n 2014, 2015 epic records, a division of sony music entertainment\n9-Jan-15\ncredit\n2:51',), ('slow down (remixes)\nselena gomez\n 2013 hollywood records, inc.\n20-Aug-13\nslow down (smash mode remix)\n5:21',), ('slow down (reggae remixes) - single\nselena gomez\n 2013 hollywood records, inc.\n20-Aug-13\nslow down (sure shot rockers reggae dub remix)\n3:15',), ('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',), (

In [193]:
print(f"The metadata is {metadata}")

The metadata is {'prompt_token_count': 25833, 'candidates_token_count': 493, 'total_token_count': 26326, 'total_calls': 9}


In [194]:
#FOR PROBLEMATIC SITUATION
solution_prompt="SELECT * FROM left_table INNER JOIN left_tableaggregateright_tableaggregate_table ON left_table.aggregate = left_tableaggregateright_tableaggregate_table.word INNER JOIN right_table ON left_tableaggregateright_tableaggregate_table.synonym = right_table.aggregate;"
required_answer=query_database(solution_prompt, printing=False)
required_answer
if answer!=required_answer:
    answer=required_answer
print(required_answer)

[('slow down (remixes)\nselena gomez\n 2013 hollywood records, inc.\n20-Aug-13\nslow down (smash mode remix)\n5:21', 'slow down (remixes)\nselena gomez\n 2013 hollywood records, inc.\n20-Aug-13\nslow down (smash mode remix)\n5:21', 'slow down remixes\nselena gomez\n (c) 2013 hollywood records, inc.\n August 20, 2013\nslow down (smash mode remix)\n5:21', 'slow down remixes\nselena gomez\n (c) 2013 hollywood records, inc.\n August 20, 2013\nslow down (smash mode remix)\n5:21'), ('slow down (reggae remixes) - single\nselena gomez\n 2013 hollywood records, inc.\n20-Aug-13\nslow down (sure shot rockers reggae dub remix)\n3:15', 'slow down (reggae remixes) - single\nselena gomez\n 2013 hollywood records, inc.\n20-Aug-13\nslow down (sure shot rockers reggae dub remix)\n3:15', 'slow down remixes\nselena gomez\n (c) 2013 hollywood records, inc.\n August 20, 2013\nslow down (smash mode remix)\n5:21', 'slow down remixes\nselena gomez\n (c) 2013 hollywood records, inc.\n August 20, 2013\nslow down

In [195]:
# Remove duplicates inside tuples and ensure unique tuples
unique_result = list(set(tuple(sorted(set(t))) for t in answer))
unique_result

[('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',
  'vhs [explicit]\nx ambassadors\n (c) 2015 kidinakorner/interscope records\n June 30, 2015\nvhs outro (interlude) [explicit]\n1:25'),
 ('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',
  'vhs [explicit]\nx ambassadors\n (c) 2015 kidinakorner/interscope records\n June 30, 2015\nsmoke (interlude) [explicit]\n0:24'),
 ('slow down (reggae remixes) - single\nselena gomez\n 2013 hollywood records, inc.\n20-Aug-13\nslow down (sure shot rockers reggae dub remix)\n3:15',
  'slow down remixes\nselena gomez\n (c) 2013 hollywood records, inc.\n August 20, 2013\nslow down (smash mode remix)\n5:21'),
 ('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',
  'vhs [explicit]\nx ambassadors\n (c) 2015 kidinakorner/interscope records\n June 30, 2015\nfirst show (interlude)\n0:11'),
 ('slow down (remixes)

In [196]:
ground_truth_answer

[('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',),
 ('title (deluxe)\nmeghan trainor\n 2014, 2015 epic records, a division of sony music entertainment\n9-Jan-15\ncredit\n2:51',),
 ('slow down (remixes)\nselena gomez\n 2013 hollywood records, inc.\n20-Aug-13\nslow down (smash mode remix)\n5:21',),
 ('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',),
 ('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',),
 ('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',),
 ('the reason - ep\nx ambassadors\n 2015 kidinakorner/interscope records\n10-Aug-15\nshining\n3:40',)]